In [1]:
import torch
from torch import nn
import  torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 读取Bert

In [2]:
from transformers import BertModel, BertTokenizer
# 这里我们调用bert-base模型，同时模型的词典经过小写处理
model_name = 'bert-base-uncased'
# 读取模型对应的tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name, cache_dir='./transformers/bert-base-uncased/')
# 载入模型
model = BertModel.from_pretrained(model_name, cache_dir='./transformers/bert-base-uncased/')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
model.to(device)
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

# 定义textCNN

In [4]:
class GlobalMaxPool1d(nn.Module):
    def __init__(self):
        super(GlobalMaxPool1d, self).__init__()
    def forward(self, x):
         # x shape: (batch_size, channel, seq_len)
        return F.max_pool1d(x, kernel_size=x.shape[2]) # shape: (batch_size, channel, 1)

In [5]:
class TextCNN(nn.Module):
    def __init__(self, embed_size, kernel_sizes, num_channels):
        super(TextCNN, self).__init__()
        # self.embedding = nn.Embedding(len(vocab), embed_size)
        # 不参与训练的嵌入层
        # self.constant_embedding = nn.Embedding(len(vocab), embed_size)
        self.dropout = nn.Dropout(0.5)
        self.decoder_1 = nn.Linear(sum(num_channels), 256)
        self.decoder_2 = nn.Linear(256, 2)
        # 时序最大池化层没有权重，所以可以共用一个实例
        self.pool = GlobalMaxPool1d()
        self.convs = nn.ModuleList()  # 创建多个一维卷积层
        
        for c, k in zip(num_channels, kernel_sizes):
            self.convs.append(nn.Conv1d(in_channels = embed_size, 
                                        out_channels = c, 
                                        kernel_size = k))

    def forward(self, inputs):
        outputs = model(inputs)[0] #shape（512， 768）
        embeddings = outputs # (batch, seq_len, embed_size)
        # 根据Conv1D要求的输入格式，将词向量维，即一维卷积层的通道维(即词向量那一维)，变换到前一维
        embeddings = embeddings.permute(0, 2, 1) # 交换维度的函数
        # 对于每个一维卷积层，在时序最大池化后会得到一个形状为(批量大小, 通道大小, 1)的
        # Tensor。使用flatten函数去掉最后一维，然后在通道维上连结
        encoding = torch.cat([self.pool(F.relu(conv(embeddings))).squeeze(-1) for conv in self.convs], dim=1)
        # 应用丢弃法后使用全连接层得到输出
        middle = self.decoder_1(self.dropout(encoding))
        
        outputs = self.decoder_2(self.dropout(F.relu(middle)))
        return outputs

## 读保存的权重

In [6]:
PATH = "./SaveModel/BertBaseEpoch10_yelp_0.96925.pth"
net = torch.load(PATH)

In [7]:
net.to(device)

TextCNN(
  (dropout): Dropout(p=0.5, inplace=False)
  (decoder_1): Linear(in_features=300, out_features=256, bias=True)
  (decoder_2): Linear(in_features=256, out_features=2, bias=True)
  (pool): GlobalMaxPool1d()
  (convs): ModuleList(
    (0): Conv1d(768, 100, kernel_size=(3,), stride=(1,))
    (1): Conv1d(768, 100, kernel_size=(4,), stride=(1,))
    (2): Conv1d(768, 100, kernel_size=(5,), stride=(1,))
  )
)

# 测试

In [8]:
def predict_sentiment(net, sentence):
    """sentence是词语的列表"""
    device = list(net.parameters())[0].device
    sentence = torch.tensor(tokenizer.encode(s, add_special_tokens=True, max_length = 512, padding='max_length', truncation=True), device=device)
    result = net(sentence.view((1, -1)))
    m = nn.Softmax(dim=1)
    print(m(result))
    label = torch.argmax(result, dim=1)
    return 'positive' if label.item() == 1 else 'negative'

In [9]:
print("请输入一句与医疗相关的英文：")
s = input() 
print(predict_sentiment(net, s))

请输入一句与医疗相关的英文：
I feel that this hospital is average and the charges are high, but the medical level is average
tensor([[0.9028, 0.0972]], device='cuda:0', grad_fn=<SoftmaxBackward>)
negative


In [13]:
print("请输入一句与医疗相关的英文：")
s = input() 
print(predict_sentiment(net, s))

请输入一句与医疗相关的英文：
This hospital is very good.
tensor([[0.0149, 0.9851]], device='cuda:0', grad_fn=<SoftmaxBackward>)
positive


In [14]:
print("请输入一句与医疗相关的英文：")
s = input() 
print(predict_sentiment(net, s))

请输入一句与医疗相关的英文：
It's too bad. It was just a minor illness. It hasn't been ranked in the number. It's getting worse and worse
tensor([[9.9991e-01, 9.0868e-05]], device='cuda:0', grad_fn=<SoftmaxBackward>)
negative


In [15]:
print("请输入一句与医疗相关的英文：")
s = input() 
print(predict_sentiment(net, s))

请输入一句与医疗相关的英文：
I never thought I would write a review for a doctor, but I feel as though Dr. Thomas Leath deserves it. Today was my first visit to this location. I had been his patient at another facility. The office staff was very nice, the nurse was sweet, and everything was clean. There was a beautiful view if the city outside the exam room window too!  Dr. Leath is most likely the best doctor I've ever seen here or in my old home town near Pittsburgh. He really listens and cares. You feel like you are working together, not being talked down to or talked at. I feel like he honestly cares about making my allergy condition better and that means a lot. If you are looking for an amazing doctor, Dr. Leath at Allergy and Asthma Associates is the place to go!
tensor([[1.6134e-17, 1.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward>)
positive
